In [1]:
%config IPCompleter.greedy=True

In [2]:
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
import os

import spacy, pickle

In [4]:
path = "./"
save_data_path = path + 'save_data/'
large_save_data_path = '/notebooks/large-storage/'
print(os.listdir(path))

['test.tsv', 'prepare-word-embedding-nlp.ipynb', 'tokenization.ipynb', 'test-batching-padding.ipynb', 'train.tsv', 'test-batching-padding-ok.ipynb', 'sampleSubmission.csv', 'save_data', '.ipynb_checkpoints', '__init__.py', 'common', 'simple-GRU-implement.ipynb']


In [5]:
max_vocab = 60000

In [6]:
UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ = "xxunk xxpad xxbos xxeos xxrep xxwrep xxup xxmaj".split()

default_spec_tok = [UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ]

In [7]:
loaded_data = pickle.load(open(save_data_path + 'pre-processed-data.pkl', 'rb'))
loaded_kaggle_test = pickle.load(open(save_data_path + 'pre-processed-kaggle-test.pkl', 'rb'))
loaded_vocab = pickle.load(open(save_data_path + 'genereated-vocab.pkl', 'rb'))

In [8]:
loaded_data.head()

,PhraseId,SentenceId,Phrase,Sentiment,Phrase_length,Tokenized_phrase,Indexed_phrase
0,1,1,A series of escapades demonstrating the adage ...,1,188,"[xxbos, a, series, of, escapades, demonstratin...","[2, 10, 341, 11, 14246, 6044, 8, 6604, 19, 64,..."
1,2,1,A series of escapades demonstrating the adage ...,2,77,"[xxbos, a, series, of, escapades, demonstratin...","[2, 10, 341, 11, 14246, 6044, 8, 6604, 19, 64,..."
2,3,1,A series,2,8,"[xxbos, a, series, xxeos]","[2, 10, 341, 3]"
3,4,1,A,2,1,"[xxbos, a, xxeos]","[2, 10, 3]"
4,5,1,series,2,6,"[xxbos, series, xxeos]","[2, 341, 3]"


In [9]:
loaded_kaggle_test.head()

,PhraseId,SentenceId,Phrase,Phrase_length,Tokenized_phrase,Indexed_phrase
0,156061,8545,An intermittently pleasing but mostly routine ...,188,"[xxbos, xxmaj, an, intermittently, pleasing, b...","[2, 7, 26, 2606, 1723, 30, 632, 1041, 409, 15, 3]"
1,156062,8545,An intermittently pleasing but mostly routine ...,77,"[xxbos, xxmaj, an, intermittently, pleasing, b...","[2, 7, 26, 2606, 1723, 30, 632, 1041, 409, 3]"
2,156063,8545,An,8,"[xxbos, xxmaj, an, xxeos]","[2, 7, 26, 3]"
3,156064,8545,intermittently pleasing but mostly routine effort,1,"[xxbos, intermittently, pleasing, but, mostly,...","[2, 2606, 1723, 30, 632, 1041, 409, 3]"
4,156065,8545,intermittently pleasing but mostly routine,6,"[xxbos, intermittently, pleasing, but, mostly,...","[2, 2606, 1723, 30, 632, 1041, 3]"


In [10]:
loaded_vocab[:10]

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxrep',
 'xxwrep',
 'xxup',
 'xxmaj',
 'the',
 ',']

In [11]:
len(loaded_vocab)

15375

In [12]:
nlp = spacy.load('en_core_web_lg')

In [13]:
count_step = 0
unknown_words = []
count_unknown = 0
for word in loaded_vocab:
    count_step += 1
    if nlp.vocab.has_vector(word) is False:
        unknown_words.append(word)
        count_unknown += 1
        

In [14]:
count_unknown

500

In [15]:
count_step

15375

In [16]:
unknown_words[:10]

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxrep',
 'xxwrep',
 'xxup',
 'xxmaj',
 '-rrb-',
 '-lrb-']

In [17]:
data_copy = nlp.vocab.vectors.data

In [18]:
data_copy.shape

(684831, 300)

In [19]:
nlp.vocab.vectors.resize((nlp.vocab.vectors.data.shape[0], nlp.vocab.vectors.data.shape[1] + len(default_spec_tok)))

[]

In [20]:
data_copy.shape

(684831, 300)

In [21]:
for row in range(0, nlp.vocab.vectors.data.shape[0]):
    nlp.vocab.vectors.data[row] = np.concatenate((data_copy[row], np.zeros(len(default_spec_tok), dtype='float32')))
    

In [22]:
nlp.vocab.vectors.data.shape

(684831, 308)

In [23]:
cat_id = nlp.vocab.strings['cat']

In [24]:
cat_id

5439657043933447811

In [25]:
nlp.vocab.vectors.key2row[cat_id]

1081

In [26]:
data_copy[1081]

array([-0.15067  , -0.024468 , -0.23368  , -0.23378  , -0.18382  ,
        0.32711  , -0.22084  , -0.28777  ,  0.12759  ,  1.1656   ,
       -0.64163  , -0.098455 , -0.62397  ,  0.010431 , -0.25653  ,
        0.31799  ,  0.037779 ,  1.1904   , -0.17714  , -0.2595   ,
       -0.31461  ,  0.038825 , -0.15713  , -0.13484  ,  0.36936  ,
       -0.30562  , -0.40619  , -0.38965  ,  0.3686   ,  0.013963 ,
       -0.6895   ,  0.004066 , -0.1367   ,  0.32564  ,  0.24688  ,
       -0.14011  ,  0.53889  , -0.80441  , -0.1777   , -0.12922  ,
        0.16303  ,  0.14917  , -0.068429 , -0.33922  ,  0.18495  ,
       -0.082544 , -0.46892  ,  0.39581  , -0.13742  , -0.35132  ,
        0.22223  , -0.144    , -0.048287 ,  0.3379   , -0.31916  ,
        0.20526  ,  0.098624 , -0.23877  ,  0.045338 ,  0.43941  ,
        0.030385 , -0.013821 , -0.093273 , -0.18178  ,  0.19438  ,
       -0.3782   ,  0.70144  ,  0.16236  ,  0.0059111,  0.024898 ,
       -0.13613  , -0.11425  , -0.31598  , -0.14209  ,  0.0281

In [27]:
nlp.vocab.vectors.data[1081]

array([-0.15067  , -0.024468 , -0.23368  , -0.23378  , -0.18382  ,
        0.32711  , -0.22084  , -0.28777  ,  0.12759  ,  1.1656   ,
       -0.64163  , -0.098455 , -0.62397  ,  0.010431 , -0.25653  ,
        0.31799  ,  0.037779 ,  1.1904   , -0.17714  , -0.2595   ,
       -0.31461  ,  0.038825 , -0.15713  , -0.13484  ,  0.36936  ,
       -0.30562  , -0.40619  , -0.38965  ,  0.3686   ,  0.013963 ,
       -0.6895   ,  0.004066 , -0.1367   ,  0.32564  ,  0.24688  ,
       -0.14011  ,  0.53889  , -0.80441  , -0.1777   , -0.12922  ,
        0.16303  ,  0.14917  , -0.068429 , -0.33922  ,  0.18495  ,
       -0.082544 , -0.46892  ,  0.39581  , -0.13742  , -0.35132  ,
        0.22223  , -0.144    , -0.048287 ,  0.3379   , -0.31916  ,
        0.20526  ,  0.098624 , -0.23877  ,  0.045338 ,  0.43941  ,
        0.030385 , -0.013821 , -0.093273 , -0.18178  ,  0.19438  ,
       -0.3782   ,  0.70144  ,  0.16236  ,  0.0059111,  0.024898 ,
       -0.13613  , -0.11425  , -0.31598  , -0.14209  ,  0.0281

In [28]:
nlp.vocab.vectors.data[684830]

array([-6.3354e-01, -1.5030e-01, -3.6161e-01, -6.3806e-02, -1.3471e-02,
        3.4250e-01,  3.3122e-01,  1.9020e-01, -3.7398e-01, -6.5865e-01,
        3.9662e-01,  1.4468e-01,  6.4382e-01, -4.9795e-01,  3.7625e-01,
       -7.4989e-02,  5.9445e-01, -6.1448e-01, -7.4406e-02,  1.3255e-01,
        4.5157e-01,  8.6841e-01, -1.1414e-01,  5.1871e-01, -1.5472e-02,
       -1.0246e+00, -1.7765e-01, -6.2875e-01,  2.6001e-01,  3.7180e-01,
        5.3249e-01,  4.3495e-01,  4.7820e-01,  2.8072e-01, -2.7053e-01,
       -1.6216e-02, -6.6976e-02, -7.9715e-02, -1.1395e+00,  8.9344e-02,
        1.6911e-01,  3.2494e-01, -2.8684e-02,  3.3086e-01, -6.6666e-02,
        7.0619e-01,  2.4593e-01,  2.4632e-01,  2.3338e-01, -3.0678e-01,
        2.0336e-01,  7.1640e-02,  2.9092e-01, -4.5740e-01, -2.1707e-02,
       -5.0103e-01,  1.1340e-02,  1.7269e-01, -2.2537e-02,  4.0978e-01,
        7.7086e-02, -5.1286e-02, -5.6078e-01, -2.1159e-02,  1.0726e-01,
        3.6532e-01,  2.6477e-01,  4.2174e-01, -2.0480e-01,  7.51

In [29]:
np.amax(nlp.vocab.vectors.data)

4.1901

In [30]:
np.amin(nlp.vocab.vectors.data)

-4.2095

In [31]:
for index in range(0, len(default_spec_tok)):
    vector_before = np.zeros(data_copy.shape[1], dtype='float32')
    vector_after = np.zeros(len(default_spec_tok), dtype='float32')
    vector_after[index] = 4.0
    nlp.vocab.set_vector(default_spec_tok[index], np.concatenate((vector_before, vector_after)))

In [32]:
default_spec_tok

['xxunk', 'xxpad', 'xxbos', 'xxeos', 'xxrep', 'xxwrep', 'xxup', 'xxmaj']

In [34]:
nlp.vocab['xxunk'].vector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [35]:
nlp.vocab.vectors.data.shape

(890280, 308)

In [36]:
nlp.vocab.vectors.data[890279]

array([-0.27837  , -0.46538  ,  0.30197  ,  0.086366 ,  0.2523   ,
        0.31446  , -0.19266  , -0.4713   , -0.46076  , -1.4654   ,
        0.13675  , -0.099137 ,  0.28635  , -0.368    ,  0.63788  ,
       -0.2872   ,  0.42602  ,  0.6683   ,  0.68467  , -0.53366  ,
       -0.85588  ,  0.42081  ,  0.68168  ,  0.030845 , -0.11942  ,
        0.28647  , -0.66554  , -0.022721 ,  0.18024  , -0.26839  ,
       -0.36913  ,  0.1685   , -0.66992  , -0.010628 ,  0.044094 ,
       -0.40207  ,  0.12269  , -0.095124 ,  0.18241  , -0.74029  ,
        0.58016  ,  0.06334  , -0.37601  , -0.38864  , -0.34038  ,
        0.20257  , -1.1577   , -0.33895  , -0.26114  , -0.076692 ,
       -0.52403  ,  0.35816  ,  0.09701  , -0.82627  , -0.25444  ,
        0.068383 ,  0.27211  , -0.53722  ,  0.26432  , -0.0084242,
        0.060024 , -0.30197  ,  0.38939  ,  0.22623  , -0.32817  ,
        0.15224  , -0.20319  , -0.31678  , -0.10334  ,  0.17426  ,
       -0.60875  , -0.15151  , -0.19635  ,  0.74664  , -0.1957

In [37]:
data_copy[890279]

IndexError: index 890279 is out of bounds for axis 0 with size 684831

In [38]:
nlp.vocab.vectors.key2row[nlp.vocab.strings['cat']]

1081

In [39]:
nlp.vocab.vectors.shape

(890280, 308)

In [40]:
nlp.to_disk(large_save_data_path + 'process-spacy-model' )

In [52]:
pickle.dump(nlp, open(large_save_data_path + 'processed-nlp.pkl', 'wb'))

In [53]:
count_step = 0
unknown_words = []
count_unknown = 0
for word in loaded_vocab:
    count_step += 1
    if nlp.vocab.has_vector(word) is False:
        unknown_words.append(word)
        count_unknown += 1

In [54]:
count_unknown

492

In [55]:
count_step

15375

In [56]:
unknown_words[:20]

['-rrb-',
 '-lrb-',
 'solondz',
 'chabrol',
 'nijinsky',
 'rohmer',
 'jaglom',
 'attal',
 'bielinsky',
 'kieslowski',
 'toback',
 'birot',
 'clockstoppers',
 'kosashvili',
 'wollter',
 'hitchcockian',
 'k-19',
 'shadyac',
 'oedekerk',
 'feardotcom']

In [57]:
nlp_origin = spacy.load('en_core_web_lg')

In [70]:
flag = True
for word in loaded_vocab:
    result_processed_nlp = nlp.vocab[word].vector
    result_origin = nlp_origin.vocab[word].vector
    diff = np.setdiff1d(result_origin, result_processed_nlp)
    if len(diff) > 0:
        print(diff)
        print("Word: " + word)
        flag = False
        break

In [71]:
flag

True